In [ ]:
from scipy.io import wavfile
fs,x = wavfile.read("Cascada - Everytime We Touch (Official Video).wav")

x = x[:,0]

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

filterFreqLp = 300
filterFreqHp = 200

soslp = signal.butter(10, filterFreqLp, 'lowpass',fs=fs,output='sos')
soshp = signal.butter(10, filterFreqHp, 'highpass',fs=fs,output='sos')
# w, h = signal.freqs(b, a)

# plt.semilogx(w, 20 * np.log10(abs(h)))
# plt.title('Butterworth filter frequency response')
# plt.xlabel('Frequency [radians / second]')
# plt.ylabel('Amplitude [dB]')
# plt.margins(0, 0.1)
# plt.grid(which='both', axis='both')
# plt.axvline(100, color='green') # cutoff frequency
# plt.show()

filtered = signal.sosfilt(soslp, x).astype(np.int16)
filtered = signal.sosfilt(soshp, filtered).astype(np.int16)

wavfile.write("outputtest.wav",fs,filtered)

In [ ]:
import plotly.graph_objs as go
# Define the length and overlap of the sliding window
window_length_sec = .1
window_overlap_sec = 0.05
window_length = int(window_length_sec * fs)
window_overlap = int(window_overlap_sec * fs)

# Calculate the power of the filtered signal using a sliding window approach
power = []
for i in range(0, len(filtered)-window_length, window_overlap):
    window = filtered[i:i+window_length]
    power.append(np.sum(window ** 2) / window_length)

# Plot the power values as a function of time
time = np.arange(len(power)) * (window_length - window_overlap) / float(fs)

powpow = np.power(power,2)

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=powpow, mode='lines'))
fig.update_layout(title='Power of Filtered Signal', xaxis_title='Time (s)', yaxis_title='Power')
fig.show()

In [ ]:
# Matched Filter Experiment: Unsucessfull
# fsS,Kick = wavfile.read("KickSample.wav")


# print(Kick.shape)

kick = filtered[int(45.5*44100):int(45.8*44100)]
kick= kick[::-1]


correlation = np.convolve(filtered,kick , mode='valid')
# correlation = signal.correlate(filtered, Kick[::-1].astype(np.int16), mode='same')

fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=correlation, mode='lines'))
fig.update_layout(title='Power of Filtered Signal', xaxis_title='Time (s)', yaxis_title='Power')
fig.show()


In [ ]:
peak = []


ispeak = False

threshold = 5_000_000

for i,sample in enumerate(powpow):
    if sample > threshold and not ispeak:
        peak += [i]
        ispeak = True
    elif sample <= threshold and ispeak:
        ispeak = False

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=powpow, mode='lines'))
fig.update_layout(title='Power of Filtered Signal', xaxis_title='Time (s)', yaxis_title='Power')

ratio = len(powpow)/len(filtered) #*(len(filtered)/fs)
# ratio = ratio*10

for i, x_line in enumerate(peak):
    x_of_line = time[x_line]
    # print(int(x_line*ratio))
    # print(x_of_line)
    fig.add_vline(x=x_of_line, line_width=3, line_dash="dash", line_color="green")

fig.show()

In [ ]:
import cv2 as cv
fps = 30

whiteFrame = np.ones((160,320,3)).astype(np.uint8)*255

greenFrame = np.zeros_like(whiteFrame)
greenFrame[:,:,1] = np.ones_like(whiteFrame[:,:,0])*255

out = cv.VideoWriter("outputAgain.mp4", cv.VideoWriter_fourcc(*'mp4v'), fps, (320, 160))

peakFrame = [int((time[pe])*fps) for pe in peak]

for frameNum in range(len(x)//fs*fps):
    # t = frameNum/fps

    if frameNum+3 in peakFrame:
        out.write(greenFrame)
    else:
        out.write(whiteFrame)

out.release()

In [ ]:


!ffmpeg -y -i outputAgain.mp4 -i "Cascada - Everytime We Touch (Official Video).wav" -map 0:v:0 -map 1:a:0 -c:a aac -b:a 192k  GreenFlashTest.mp4